In [1]:
import pandas as pd
import gspread
import os
import shutil
import time

#Pré Processamento

start = time.time()

#Passo 1: Pega os exercicios com status Produzido na planilha:

codigo_planilha = '1109p2Juec5nKNSZj0DyyN7MVH1tqMRQimE1gDDZGYgk'
local_token = r'C:\Users\Lenovo\Responde Ai\Upload Questoes\Script\token.json'
aba_planilha = 'Consolidados'

gc = gspread.service_account(filename = local_token)
sh = gc.open_by_key(codigo_planilha)
ws = sh.worksheet(aba_planilha)
coluna_status = ws.find('Status').col
coluna_ids = ws.find('ID na content').col

dados = ws.get_all_values()

headers = dados.pop(5)

tabela = pd.DataFrame(dados, columns=headers)

produzidos = tabela[tabela['Status'] == 'Produzido']['ID Diretório Arquivo']

lista_produzidos = []
encontrados = []
n_encontrados = []
maior_1mb = []
menor_1mb = []

#Passo 2: Verifica se o arquivo existe na pasta de entregas, e verifica tbm se o arquivo possui mais de 1MB

for id in produzidos:
    
    id = str(id)
    if os.path.exists(id):
        encontrados.append(id)
        tamanho = os.path.getsize(id)
        if tamanho/1000 > 1000:
            maior_1mb.append(id)
        else:
            menor_1mb.append(id)
        
    else:
        n_encontrados.append(id)
        
    lista_produzidos.append(id)

for qst in menor_1mb:
    qst_caminho = '\\'.join(qst.split('\\')[:-1])
    destino = qst_caminho.replace('Provas e Listas', 'Upload')
    if not os.path.exists(destino):
        os.makedirs(destino)
    shutil.copy(qst, destino)

total_encontrados = len(encontrados)
total_n_encontrados = len(n_encontrados)
total_produzido = total_encontrados + total_n_encontrados
total_maior_1mb = len(maior_1mb)
total_menor_1mb = len(menor_1mb)

print('Pré-Processamento finalizado')

Pré-Processamento finalizado


In [2]:
#Processamento

import pyautogui as pa
import pyperclip as pc
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd
import gspread
import os
from pathlib import Path

pasta_arquivos_upload = r'G:\.shortcut-targets-by-id\1XWBk8HfBV1zIvxD2bbElLvXLDCnii-k9\Conteúdo - Produção\Entregas\Upload'
indice_exercicio = 67998 # ver com Kuritza se tem forma de obter o último índice na content
email = 'felipehfernandes1807@gmail.com'
pword = 'felipe12'

# Função que seleciona o exercicio para upar
def select_info(info):
    pc.copy(info)
    pa.hotkey("ctrl","v")
    pa.press("enter")

pa.alert('O Código vai começar a rodar, por favor não mexer no computador')
pa.pause = 0.5

# Passo 1: Entrar no navegador, e logar na content:

navegador = webdriver.Chrome()
navegador.get("https://content.respondeai.com.br/")
pa.hotkey("win","up")

sucessos = []
erros_upload = []
nao_upados_nao_encontrados_erros = []
upados_nao_encontrados = []
caixa_email = '//*[@id="inputEmail"]'
caixa_senha = '//*[@id="inputPassword"]'
botao_guias_personalizados = '//*[@id="bs-navbar-collapse"]/ul[1]/li[2]/a'
seletor_faculdade = '//*[@id="select2-select-universities-container"]'
seletor_campus = '//*[@id="select2-select-university-campi-container"]'
seletor_disciplina = '//*[@id="select2-select-disciplines-container"]'
seletor_categoria_lista = '//*[@id="select2-select-list-categories-container"]'
seletor_lista = '//*[@id="select2-select-lists-container"]'
seletor_upload_exercicio = '//*[@id="file-upload-trigger"]'
seletor_upload_erro = '/html/body/div/div[2]/form/div[3]/div[2]/strong'




navegador.find_element('xpath', caixa_email).send_keys(email)
navegador.find_element('xpath', caixa_senha).send_keys(pword)
navegador.find_element('xpath', caixa_senha).send_keys(Keys.ENTER)



#Passo 2: Selecionar a faculdade, campus, categoria de lista e lista:

navegador.find_element('xpath',botao_guias_personalizados).click()

# Selecionar Faculdade:

os.chdir(pasta_arquivos_upload)
caminho = Path.cwd()
faculdades = os.listdir()
faculdades.remove('desktop.ini')


tentativas = 0
proxima = False
while not proxima:

    # Escolher Faculdade
    for faculdade in faculdades:
        if faculdade != 'desktop.ini':         
            navegador.find_element('xpath', seletor_faculdade).click()
            select_info(faculdade)
            os.chdir(pasta_arquivos_upload)
            caminho = Path.cwd()
            caminho = os.path.join(caminho, faculdade)
            os.chdir(caminho)

    # Escolher Campus

        tcampus = os.listdir()
        tcampus.remove('desktop.ini')
        for campus in tcampus:
            if campus != 'desktop.ini':
                navegador.find_element('xpath', seletor_campus).click()
                select_info(campus)
                os.chdir(pasta_arquivos_upload)
                caminho = Path.cwd()
                caminho = os.path.join(caminho, faculdade, campus)
                os.chdir(caminho)

    # Escolher Disciplina

            disciplinas = os.listdir()
            disciplinas.remove('desktop.ini')

            for disciplina in disciplinas:
                if disciplina != 'desktop.ini':
                    navegador.find_element('xpath', seletor_disciplina).click()
                    select_info(disciplina)
                    os.chdir(pasta_arquivos_upload)
                    caminho = Path.cwd()
                    caminho = os.path.join(caminho, faculdade, campus, disciplina)
                    os.chdir(caminho)

    # Escolher Categoria de Lista

                categorias = os.listdir()
                categorias.remove('desktop.ini')

                for categoria in categorias:
                    if categoria != 'desktop.ini':
                        navegador.find_element('xpath', seletor_categoria_lista).click()
                        select_info(categoria)
                        os.chdir(pasta_arquivos_upload)
                        caminho = Path.cwd()
                        caminho = os.path.join(caminho, faculdade,campus,disciplina,categoria)
                        os.chdir(caminho)

    # Escolher Lista

                    listas = os.listdir()
                    listas.remove('desktop.ini')

                    for lista in listas:
                        if lista != 'desktop.ini':
                            navegador.find_element('xpath', seletor_lista).click()
                            select_info(lista)
                            os.chdir(pasta_arquivos_upload)
                            caminho = Path.cwd()
                            caminho = os.path.join(caminho, faculdade, campus, disciplina, categoria, lista)
                            os.chdir(caminho)

                        exercicios = []
                        
    # Upar questão
    
                        for root, subfolder, files in os.walk(caminho):
                            for file in files:
                                if file != 'desktop.ini':
                                    os.chdir(pasta_arquivos_upload)
                                    caminho = Path.cwd()
                                    caminho = os.path.join(caminho, faculdade, campus, disciplina, categoria, lista, file)
                                    exercicios.append(caminho)


                        for exercicio in exercicios:



                            #tenta selecionar o botão de upar
                            
                            try:
                                navegador.find_element('xpath', seletor_upload_exercicio).click()
                                time.sleep(1)
                                pc.copy(exercicio)
                                pa.hotkey("ctrl","v")
                                pa.press("enter")
                                time.sleep(3)
                                                               
                                #verifica a mensagem de sucesso
                                
                                try:
                                    msg_sucesso = navegador.find_element('xpath',seletor_upload_erro).get_attribute('value')
                                    sucessos.append(exercicio)
                                    exercicio = exercicio.replace('Upload', 'Provas e Listas')
                                    encontrado = ws.find(exercicio)
                                    if encontrado is not None: 
                                        cell_row = encontrado.row
                                        ws.update_cell(cell_row,coluna_status,"No site")
                                        ws.update_cell(cell_row,coluna_ids, indice_exercicio)
                                        indice_exercicio = indice_exercicio + 1
                                    elif encontrado is None:
                                        upados_nao_encontrados.append(exercicio)
                                    else:
                                        continue

                                except:

                                    erros_upload.append(exercicio)
                                    exercicio = exercicio.replace('Upload', 'Provas e Listas')
                                    encontrado = ws.find(exercicio)
                                    if encontrado is not None:
                                        cell_row = encontrado.row
                                        ws.update_cell(cell_row,coluna_status,"Erro de upload")
                                    elif encontrado is None:
                                        nao_upados_nao_encontrados_erros.append(exercicio)
                                    else:
                                        continue

                                proxima = True
                                
                            except:
                                exercicio = exercicio.replace('Upload', 'Provas e Listas')
                                encontrado = ws.find(exercicio)
                                if encontrado is not None:
                                    cell_row = encontrado.row
                                    ws.update_cell(cell_row,coluna_status,"Erro de upload")
                                elif encontrado is None:
                                    nao_upados_nao_encontrados_erros.append(exercicio)
                                navegador.get("https://content.respondeai.com.br/disciplines/file-upload")
                                tentativas = tentativas + 1
                                if tentativas >= 2:
                                    exercicio = exercicio.replace('Provas e Listas', 'Upload')
                                    erros_upload.append(exercicio)
                                    sucesso = True
                                    
os.chdir(pasta_arquivos_upload)

with open("questoes_produzidas_nao_encontradas_drive.txt", "w") as arquivo:
    for item in n_encontrados:
        item = item.replace('Provas e Listas', 'Upload')
        arquivo.write(item.replace(pasta_arquivos_upload, '')[1:] + "\n")
with open("questoes_problemas_tamanho.txt", "w") as arquivo:
    for item in maior_1mb:
        item = item.replace('Provas e Listas', 'Upload')
        arquivo.write(item.replace(pasta_arquivos_upload, '')[1:] + "\n")
with open("questoes_problemas_upload.txt", "w") as arquivo:
    for item in erros_upload:
        item = item.replace('Provas e Listas', 'Upload')
        arquivo.write(item.replace(pasta_arquivos_upload, '')[1:] + "\n")
                                    
                                    
end = time.time()
total_questoes = len(erros_upload)+len(sucessos)
total_sucessos = len(sucessos)
total_erros = len(erros_upload)
tempo_total = end-start
tempo_total_minutos = (end-start)/60
media_tempo_questao = tempo_total/total_questoes



        

print("Upload Concluído")
print(" ")
print(" ")
print("1. Relatório Pré Processamento")
print(" ")
print(f"Foram encontrados {total_produzido} exercícios com o Status Produzido na planilha.")
print(f"Desses exercícios, {total_n_encontrados} não foram encontradas na pasta do Drive - Entregas.")
print(f"As outras {total_encontrados} que foram encontradas, {total_maior_1mb} estavam com tamanho maior que 1MB")
print(" ")
print("2. Relatório Processamento")
print(" ")
print(f"Dos {total_menor_1mb} exercícios dentro dos parâmetros permitidos, {total_sucessos} foram upados com sucesso.")
print(f"Por outro lado, {total_erros} exercícios tiveram problemas durante o upload, como formatação, categorias incompatíveis ou inexistentes")
print(" ")
print("3. Resultados")
print(" ")
print(f"Dos {total_questoes} exercícios processados, levamos cerca de {tempo_total_minutos} minutos para upar tudo, uma media de {media_tempo_questao} segundos por exercício")
print(f"O último exercício upado possuía o id: {indice_exercicio}")
print(" ")
print("Os exercícios com problemas podem ser encontrados nos seguintes arquivos:")
print("questoes_produzidas_nao_encontradas_drive.txt")
print("questoes_problemas_tamanho.txt")
print("questoes_problemas_upload.txt")

pa.alert('Upload finalizado :)')

==================UPLOAD CONCLUIDO==================
 
1. Relatório Pré Processamento
 
Foram encontrados 171 exercícios com o Status Produzido na planilha.
Desses exercícios, 14 não foram encontradas na pasta do Drive - Entregas.
As outras 157 que foram encontradas, 0 estavam com tamanho maior que 1MB
 
2. Relatório Processamento
 
Dos 157 exercícios dentro dos parâmetros permitidos, 155 foram upados com sucesso.
Por outro lado, 2 exercícios tiveram problemas durante o upload, como formatação, categorias incompatíveis ou inexistentes
 
3. Resultados
 
Dos 157 exercícios processados, levamos cerca de 25.33743985891342 minutos para upar tudo, uma media de 9.683098035253536 segundos por exercício
O último exercício upado possuía o id: 67997
 
Os exercícios com problemas podem ser encontrados nos seguintes arquivos:
questoes_produzidas_nao_encontradas_drive.txt
questoes_problemas_tamanho.txt
questoes_problemas_upload.txt


'OK'